In [3]:
import numpy as np
import math
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
#from ipywidgets import Layout
import mplcursors
from random import randrange
import openbabel as ob
import nglview as nv
from nglview.viewer_control import ViewerControl

In [4]:
from load_data import load_file
from helper_functions import calc_scaling,symmetrize,single_rotD,single_rotP,unit_rot
from calc_average import full_average,full_average_IR,full_average_R,full_average_R_orth,numerical_sector_average
from calc_single import oriented_IR,oriented_R,single_rot_IR,single_rot_R,single_polar_IR,single_polar_R
from plotting_functions import polar_plot

In [5]:
#filename="freq-19813-90-2.out"   # less accurate
filename="freq-19813-90-2.fchk"   # more accurate
fr,Z,Q,D,P0,nat,aniso=load_file(filename)
nmodes=len(fr)

Loaded  freq-19813-90-2.fchk


In [6]:
# parameters
dtheta=0.02
axes=[["x",1], ["y",2] ,["z",3]]

phys_params=dict(laser =785, #633, 
                 T = 298.15
                 )

Lm=1
# calculate intensity scaling factors
v0= math.pow(10, 7)/phys_params['laser']

scalingIR,scaling,scalingexp, scalingpolar= calc_scaling(phys_params['T'])
pi=math.pi
torad=2*pi/360

In [7]:
# convert to mol format for displaying vibrations
def convert_to_mol(filename,molfilename):
    #print("conv")
    obConversion = ob.OBConversion()
    obConversion.SetInAndOutFormats("out", "mol") 
    mol = ob.OBMol()
    obConversion.ReadFile(mol, filename)
    outMDL = obConversion.WriteString(mol)
    molcoords  = [[atom.GetX(),atom.GetY(),atom.GetZ()] for atom in ob.OBMolAtomIter(mol)] 

    obConversion.WriteFile(mol,molfilename)
    return molcoords
    

outfile="freq-19813-90-2.out"
molfile="freq-19813-90-2.mol"
molcoords=convert_to_mol(outfile,molfile)

C:\Users\proelli\Anaconda3\lib\openbabel\__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')


In [11]:
%matplotlib widget
# create polar plot
def plot_single_mode(mode,rb_type_study,rb_polar_plan,rb_ir_beam,rb_rin_beam,rb_rout_beam):
    m=mode-1 # Python starts from 0
    
    # Display molecule and chosen vibration
    view = nv.NGLWidget()
    comp = view.add_component(molfile) 
    control = ViewerControl(view=view)
  #  control.spin([1,0,0],-90*torad)
  #  control.spin([0,0,1],40*torad)
    sc=3.5  # scale arrows for vibration
    for at in range(nat):
        displ1=list(molcoords[at]-sc*Q[m,at])
        displ2=list(molcoords[at]+sc*Q[m,at])
        view.shape.add_arrow(displ1, displ2, [ 0, 0, 0 ], 0.2)
    # show axes
    view.shape.add_arrow([-6,-6,-6], [-4,-6,-6], [ 0, 0, 1 ], 0.2, 'x')
    view.shape.add_arrow([-6,-6,-6], [-6,-4,-6], [ 1, 0, 0 ], 0.2, 'y')
    view.shape.add_arrow([-6,-6,-6], [-6,-6,-4], [ 0, 0.8, 0.2 ], 0.2, 'z')
    view.shape.add('text', [-3.5,-6,-6], [ 0, 0, 1 ], 3, 'x')
    view.shape.add('text', [-6, -3.5, -6], [ 1, 0, 0 ], 3, 'y')
    view.shape.add('text', [-6, -6, -3.5], [ 0, 0.8, 0.2 ], 3, 'z')
    display(view)
    
    print('Click to create cursor, Right-click to remove it')
    
    #P=symmetrize(P0[m,:,:])

    # Calculate frequency-dependent scaling factors
    # Usual Stokes for thermal population
    #scalingR=Lm*scaling* math.pow(v0 - fr[m], 4) / (
    #        fr[m] * (1 - math.exp(scalingexp * fr[m]))) 
    # Usual anti-Stokes for thermal population
    #scalingaR=Lm*scaling* math.pow(v0 + fr[m], 4) / fr[m] *(
    #        1/(-1+math.exp(-scalingexp * fr[m]))) # 
    # For THOR: anti-Stokes without population
    #scalingTHOR=Lm*scaling* math.pow(v0 + fr[m], 4) / fr[m]  
    
    theta = np.arange(0, 2, dtheta)*pi
    nb_theta=len(theta)
    I=np.zeros(nb_theta)
    proj=rb_polar_plan
    e=np.array([0.,0.,1.0])

    for t in range (0,nb_theta) :
        r=unit_rot(proj,theta[t])
        if rb_type_study==1 :
            I[t]=scalingIR*single_polar_IR(D[m,:],r,e)
        elif rb_type_study==2 :
            I[t]=scalingpolar*single_polar_R(P0[m,:,:],r,e)
        elif rb_type_study==3 :
            I[t]=scalingIR*scalingpolar*single_polar_IR(D[m,:],r,e)*single_polar_R(P0[m,:,:],r,e)
    
    line=polar_plot(theta,I)
    c1 = mplcursors.cursor(line)
    @c1.connect("add")
    def _(sel):
        sel.annotation.get_bbox_patch().set(fc="white")
        sel.annotation.arrow_patch.set(arrowstyle="simple", fc="white", alpha=.5)

    ax=plt.gca()
    ax.set_title("Vibrational mode {}".format(mode),pad=20)
    plt.tight_layout()
    plt.show()

In [12]:
from ipywidgets import HBox, Layout,VBox

studies=[["IR absorption",1], ["Raman scattering",2] ,["Conversion",3]]
planes=[["x-y plane",1], ["x-z plane",2] ,["y-z plane",3]]

interactive_plot = interactive(plot_single_mode, {'manual': True}, 
         mode=widgets.BoundedIntText(value=1,min=1,max=nmodes,description='Mode:',disabled=False),
         rb_type_study=widgets.RadioButtons(
                        options=studies,
                        value=1,
                        description='Type of study:',
                        style={'description_width': 'initial'},
                    #    layout={'width': 'max-content'}, # If the items' names are long
                        disabled=False
                    ),
         rb_polar_plan=widgets.RadioButtons(
                        options=planes,
                        value=2,
                        description='Projection plane:',
                        style={'description_width': 'initial'},
                    #    layout={'width': 'max-content'}, # If the items' names are long
                        disabled=False
                    ),
         rb_ir_beam=widgets.RadioButtons(
                        options=axes,
                        value=3,
                        description='IR beam:',
                        style={'description_width': 'initial'},
                        layout={'width': '30%'}, # If the items' names are long
                        disabled=True
                    ),
        rb_rin_beam=widgets.RadioButtons(
                        options=axes,
                        value=3,
                        description='Raman (in):',
                        style={'description_width': 'initial'},
                        layout={'width': '30%'}, # If the items' names are long
                        disabled=True
                    ),
        rb_rout_beam=widgets.RadioButtons(
                        options=axes,
                        value=3,
                        description='Raman (out):',
                        style={'description_width': 'initial'},
                        layout={'width': '30%'}, # If the items' names are long
                        disabled=True
                    )

        );

output = interactive_plot.children[-1]
controls = HBox(interactive_plot.children[:-1], layout = Layout(flex_flow='row wrap')) #'column'))
output.layout.height = '800px'

display(VBox([controls,output]))